In [275]:
from __future__ import division
from __future__ import print_function
import hypertools as hyp
import seaborn as sns
from datetime import datetime

import numpy as np 
import pandas as pd

# from pylab import plot, show, figure, imshow
# %matplotlib notebook
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')



import os

import pickle

from collections import OrderedDict

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler, scale
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV


import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline




In [2]:
#Constant Variables python
# kGroup = sys.argv[1]
# kInstrument = sys.argv[1]
# kSampleRate = 44100
# kN = 100
# kType = 'mono'
# kFeatures = 1812 #Find a way to not hard-code this.     
# path = './Music/Data/MedleyDB/Features/%s/%s/' % (sys.argv[2], kType)

kInstruments = ['bass3', 'guitar5', 'vocal2', 'keys-all3']
kLabels = ['b','g','v','k']
kGroup = 'banjo'
kInstrument = 'banjo'
kSampleRate = 44100
kType = 'mono'
# path = './Music/Data/MedleyDB/Features/%s/%s/' % (sys.argv[2], kType)
path1 = '/homes/mamr3/Stage - 1/Music/Data/MedleyDB/Features/Analysis/' 
path2 = '/homes/mamr3/Stage - 1/Music/Data/MedleyDB/Features/Regression/' 
path3 = '/homes/mamr3/Stage - 1/Music/Data/MedleyDB/Features/no_boot_nw_2000/mono/' 



def dumpPickle(d, name, _path = path1):
#  path = './Music/Data/MedleyDB/Features/z_Arrays/%s/' % (kType)

  with open(_path + name, 'wb') as output:
    # Pickle dictionary using protocol 0.
    pickle.dump(d, output)
  print('%s Saved' % (name))
  
def loadPickle(name,_path = path1):  
#  path = './Music/Data/MedleyDB/Features/z_Arrays/%s/' % (kType)
  # load data from pkl file
  with open(_path + name, "rb") as fp:
      loaded_data1 = pickle.load(fp)
  
  print('%s loaded, %s ' % (name, type(loaded_data1)))
  
  return loaded_data1


In [249]:
#%%
#Only if you initialized Variables.

# First time I took the files from several folders: 

# X_All = loadPickle('X.pkl', _path = path2)
# Y_All = loadPickle('y.pkl', _path = path2)
# XRaw = loadPickle('XRaw.pkl', _path = path3)
# XStem = loadPickle('XStem.pkl', _path = path3)
# yRaw = loadPickle('yRaw.pkl', _path = path3)
# yStem = loadPickle('yStem.pkl', _path = path3)
# gNameFeatures = loadPickle('gNameFeatures.pkl', _path = path1)
# gPredictionIdx = loadPickle('gPredictionIdx.pkl', _path = path1)
# GPredictionFeatures = loadPickle('gPredictionFeatures.pkl', _path = path1)

p = '/homes/mamr3/Stage - 1/Music/Data/MedleyDB/Features/Regression/' 

df_Raw = loadPickle('df_Raw.pkl', _path = p)
X_train = loadPickle('X_train.pkl', _path = p)
X_test = loadPickle('X_test.pkl', _path = p)
Xs_train = loadPickle('Xs_train.pkl', _path = p)
Xs_test = loadPickle('Xs_test.pkl', _path = p)
Xm_train = loadPickle('Xm_train.pkl', _path = p)
Xm_test = loadPickle('Xm_test.pkl', _path = p)
df_Stem = loadPickle('df_Stem.pkl', _path = p)
y_train = loadPickle('y_train.pkl', _path = p)
y_test = loadPickle('y_test.pkl', _path = p)

X_All = loadPickle('X_All.pkl', _path = p)
Y_All = loadPickle('y_All.pkl', _path = p)
X_Raw = loadPickle('X_Raw.pkl', _path = p)
X_Stem = loadPickle('X_Stem.pkl', _path = p)

gNameFeatures = loadPickle('gNameFeatures.pkl', _path = p)
gPredictionIdx = loadPickle('gPredictionIdx.pkl', _path = p)
GPredictionFeatures = loadPickle('gPredictionFeatures.pkl', _path = p)

df_Raw.pkl loaded, <class 'collections.OrderedDict'> 
X_train.pkl loaded, <class 'collections.OrderedDict'> 
X_test.pkl loaded, <class 'collections.OrderedDict'> 
Xs_train.pkl loaded, <class 'collections.OrderedDict'> 
Xs_test.pkl loaded, <class 'collections.OrderedDict'> 
Xm_train.pkl loaded, <class 'collections.OrderedDict'> 
Xm_test.pkl loaded, <class 'collections.OrderedDict'> 
df_Stem.pkl loaded, <class 'collections.OrderedDict'> 
y_train.pkl loaded, <class 'collections.OrderedDict'> 
y_test.pkl loaded, <class 'collections.OrderedDict'> 
X_All.pkl loaded, <type 'dict'> 
y_All.pkl loaded, <type 'dict'> 
X_Raw.pkl loaded, <class 'collections.OrderedDict'> 
X_Stem.pkl loaded, <class 'collections.OrderedDict'> 
gNameFeatures.pkl loaded, <class 'collections.OrderedDict'> 
gPredictionIdx.pkl loaded, <class 'collections.OrderedDict'> 
gPredictionFeatures.pkl loaded, <class 'collections.OrderedDict'> 


In [234]:
#Gets dataframes of Predicted Feature values.

def getDFfromPickle(dataX):
    XPrediction = OrderedDict()
    gPredictionFeatures = OrderedDict()
    for k in kInstruments:
        #print '\n %s \n' % k.upper()        
        _A = dataX[k[0]][:]
        Xpreselected = [] 
        for i in gPredictionIdx[k]:

            Xpreselected.append(_A[:,i])

        XPrediction[k] = np.asarray(Xpreselected).T 

        x = []
        for f in GPredictionFeatures[k]:
            x.append('.'.join(f.split('.')[1::]))
        gPredictionFeatures[k] = x

    dfBass = pd.DataFrame(XPrediction[kInstruments[0]], columns = gPredictionFeatures[kInstruments[0]])
    dfGuitar = pd.DataFrame(XPrediction[kInstruments[1]], columns = gPredictionFeatures[kInstruments[1]])
    dfVocal = pd.DataFrame(XPrediction[kInstruments[2]], columns = gPredictionFeatures[kInstruments[2]])
    dfKeys = pd.DataFrame(XPrediction[kInstruments[3]], columns = gPredictionFeatures[kInstruments[3]])
    
    df = OrderedDict()
    df[kLabels[0]] = dfBass
    df[kLabels[1]] = dfGuitar
    df[kLabels[2]] = dfVocal
    df[kLabels[3]] = dfKeys
    
    return df

def getNPfromDF(df_R, df_S, trainSize = 0.9):
    
    A = OrderedDict()
    A_train = OrderedDict()
    A_test = OrderedDict() 
    As = OrderedDict()
    As_train = OrderedDict()
    As_test = OrderedDict() 
    Am = OrderedDict()
    Am_train = OrderedDict()
    Am_test = OrderedDict() 
    b = OrderedDict()
    b_train = OrderedDict()
    b_test = OrderedDict()
    
    
    
    
    for k in kLabels:
        
        df = df_Raw[k]
        df = df[np.abs(df-df.mean())<=(3*df.std())]
        idx = []
        for i in df.columns:
            a = pd.notnull(df[i])
            try:
                idx.append(np.where( a == False )[0][0])
                idx.append(np.where( a == False )[0][1])
            except:
                pass
        idx = list(set(idx))
        df.drop(df.index[idx], inplace=True)

        norm_df=(df-df.mean())/df.std()
        minmax_df=(df-df.min())/(df.max()-df.min())
        
        A[k] = pd.DataFrame.as_matrix(df)
        As[k] = pd.DataFrame.as_matrix(norm_df)
        Am[k] = pd.DataFrame.as_matrix(minmax_df)
        df2 = df_S[k]
        df2.drop(df2.index[idx], inplace=True)
        b[k] = pd.DataFrame.as_matrix(df2)
        A_train[k], A_test[k], As_train[k], As_test[k], Am_train[k], Am_test[k],b_train[k], b_test[k] = train_test_split(A[k],
                                                                                                                         As[k],
                                                                                                                         Am[k],
                                                                                                                         b[k],
                                                                                                                         train_size=trainSize,
                                                                                                                         random_state=42)
    return A_train, A_test, As_train, As_test, Am_train, Am_test, b_train, b_test
    







In [237]:
# X_Raw = OrderedDict()
# X_Stem = OrderedDict()
# X_Raw['b'] = XRaw['electric bass']
# X_Raw['g'] = np.vstack((XRaw['acoustic guitar'], 
#                         XRaw['clean electric guitar'], 
#                         XRaw['distorted electric guitar'], 
#                         XRaw['banjo']))
# X_Raw['v'] = np.vstack((XRaw['male singer'], 
#                         XRaw['female singer'], 
#                         XRaw['male rapper']))
# X_Raw['k'] = np.vstack((XRaw['piano'], 
#                         XRaw['synthesizer'], 
#                         XRaw['tack piano'], 
#                         XRaw['electric piano']))
# X_Stem['b'] = XStem['electric bass']
# X_Stem['g'] = np.vstack((XStem['acoustic guitar'], 
#                         XStem['clean electric guitar'], 
#                         XStem['distorted electric guitar'], 
#                         XStem['banjo']))
# X_Stem['v'] = np.vstack((XStem['male singer'], 
#                         XStem['female singer'], 
#                         XStem['male rapper']))
# X_Stem['k'] = np.vstack((XStem['piano'], 
#                         XStem['synthesizer'], 
#                         XStem['tack piano'], 
#                         XStem['electric piano']))



# df_Raw = getDFfromPickle(X_Raw)
# df_Stem = getDFfromPickle(X_Stem)

# X_train, X_test, Xs_train, Xs_test, Xm_train, Xm_test, y_train, y_test = getNPfromDF(df_Raw, df_Stem, trainSize = 0.9)


In [277]:
# p = '/homes/mamr3/Stage - 1/Music/Data/MedleyDB/Features/Regression/' 

# dumpPickle(df_Raw, 'df_Raw.pkl', _path = p)
# dumpPickle(df_Stem, 'df_Stem.pkl', _path = p)
# dumpPickle(X_train, 'X_train.pkl', _path = p)
# dumpPickle(X_test, 'X_test.pkl', _path = p)
# dumpPickle(Xs_train, 'Xs_train.pkl', _path = p)
# dumpPickle(Xs_test, 'Xs_test.pkl', _path = p)
# dumpPickle(Xm_train, 'Xm_train.pkl', _path = p)
# dumpPickle(Xm_test, 'Xm_test.pkl', _path = p)
# dumpPickle(df_Stem, 'df_Stem.pkl', _path = p)
# dumpPickle(y_train, 'y_train.pkl', _path = p)
# dumpPickle(y_test, 'y_test.pkl', _path = p)

# dumpPickle(X_All, 'X_All.pkl', _path = p)
# dumpPickle(Y_All, 'y_All.pkl', _path = p)
# dumpPickle(X_Raw, 'X_Raw.pkl', _path = p)
# dumpPickle(X_Stem, 'X_Stem.pkl', _path = p)

# dumpPickle(gNameFeatures, 'gNameFeatures.pkl', _path = p)
# dumpPickle(gPredictionIdx, 'gPredictionIdx.pkl', _path = p)
# dumpPickle(GPredictionFeatures, 'gPredictionFeatures.pkl', _path = p)


In [273]:
# define base model
def model_1(neurons=32, init_mode='zero', act_function='selu', dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(Dense(neurons*2, input_dim=4, 
                    kernel_initializer=init_mode, 
                    activation=act_function, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, kernel_initializer=init_mode, activation=act_function,
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, kernel_initializer=init_mode))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model
# define base model

def model_2(neurons=32,init_mode='zero', act_function='selu', dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(Dense(neurons*2, input_dim=4, 
                    kernel_initializer=init_mode, 
                    activation=act_function, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, kernel_initializer=init_mode, activation=act_function,
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(6, kernel_initializer=init_mode))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model

In [252]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
# estimator_b = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)
# estimator_bRS = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)
# estimator_bS = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)
# estimator_g = KerasRegressor(build_fn=model_2, nb_epoch=100, batch_size=5, verbose=0)
# estimator_gRS = KerasRegressor(build_fn=model_2, nb_epoch=100, batch_size=5, verbose=0)
# estimator_gS = KerasRegressor(build_fn=model_2, nb_epoch=100, batch_size=5, verbose=0)
# estimator_k = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)
# estimator_kRS = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)
# estimator_kS = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)
# estimator_v = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)
# estimator_vRS = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)
# estimator_vS = KerasRegressor(build_fn=model_1, nb_epoch=100, batch_size=5, verbose=0)

In [253]:
# label = 'b'
# A_train = (X_train[label])
# b_train = (y_train[label])
# b_test = (y_test[label])
# A_test = (X_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_b, A_train, b_train, cv=kfold)
# print("Results: %.2f (%.2f) MAE" % (results.mean(), results.std()))


# A_train = (Xs_train[label])
# A_test = (Xs_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_bRS, A_train, b_train, cv=kfold)
# print("Results - S: %.2f (%.2f) MAE" % (results.mean(), results.std()))


# A_train = (Xm_train[label])
# A_test = (Xm_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_bS, A_train, b_train, cv=kfold)
# print("Results - M: %.2f (%.2f) MAE" % (results.mean(), results.std()))

Results: 3.96 (0.07) MAE
Results - S: 3.96 (0.07) MAE
Results - M: 3.96 (0.07) MAE


In [254]:
# label = 'g'
# A_train = (X_train[label])
# b_train = (y_train[label])
# b_test = (y_test[label])
# A_test = (X_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_g, A_train, b_train, cv=kfold)
# print("Results: %.2f (%.2f) MAE" % (results.mean(), results.std()))


# A_train = (Xs_train[label])
# A_test = (Xs_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_gRS, A_train, b_train, cv=kfold)
# print("Results - RS: %.2f (%.2f) MAE" % (results.mean(), results.std()))


# A_train = (Xm_train[label])
# A_test = (Xm_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_gS, A_train, b_train, cv=kfold)
# print("Results - S: %.2f (%.2f) MAE" % (results.mean(), results.std()))

Results: 1.33 (0.05) MAE
Results - RS: 1.33 (0.05) MAE
Results - S: 1.33 (0.05) MAE


In [267]:
# label = 'k'
# A_train = (X_train[label])
# b_train = (y_train[label])
# b_test = (y_test[label])
# A_test = (X_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_k, A_train, b_train, cv=kfold)
# print("Results: %.2f (%.2f) MAE" % (results.mean(), results.std()))


# A_train = (Xs_train[label])
# A_test = (Xs_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_kRS, A_train, b_train, cv=kfold)
# print("Results - RS: %.2f (%.2f) MAE" % (results.mean(), results.std()))


# A_train = (Xm_train[label])
# A_test = (Xm_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_kS, A_train, b_train, cv=kfold)
# print("Results - S: %.2f (%.2f) MAE" % (results.mean(), results.std()))


Results: 7.94 (0.42) MAE
Results - RS: 7.94 (0.42) MAE
Results - S: 7.94 (0.42) MAE


In [268]:
# label = 'v'
# A_train = (X_train[label])
# b_train = (y_train[label])
# b_test = (y_test[label])
# A_test = (X_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_v, A_train, b_train, cv=kfold)
# print("Results: %.2f (%.2f) MAE" % (results.mean(), results.std()))


# A_train = (Xs_train[label])
# A_test = (Xs_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_vRS, A_train, b_train, cv=kfold)
# print("Results - RS: %.2f (%.2f) MAE" % (results.mean(), results.std()))


# A_train = (Xm_train[label])
# A_test = (Xm_test[label])

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator_vS, A_train, b_train, cv=kfold)
# print("Results - S: %.2f (%.2f) MAE" % (results.mean(), results.std()))

Results: 2.45 (0.46) MAE
Results - RS: 2.45 (0.46) MAE
Results - S: 2.45 (0.46) MAE


In [ ]:

label = sys.argv[1]
A_train = (X_train[label])
b_train = (y_train[label])
b_test = (y_test[label])
A_test = (X_test[label])


batch_size = [10, 20, 40, 50, 60, 70]
epochs = [50, 100, 150, 200]
init_mode = ['zero','uniform','lecun_uniform', 'normal']
activation = ['selu','relu','elu','tanh','sigmoid']
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4]
neurons = [8, 16, 32, 64, 128]


est = KerasRegressor(build_fn=model_1, verbose=0)


param_grid = dict(neurons=neurons,
                  batch_size=batch_size,
                  epochs=epochs,
                  dropout_rate=dropout_rate,
                  weight_constraint=weight_constraint,
                  init_mode=init_mode,
                  act_function=activation)

grid = GridSearchCV(estimator=est, param_grid=param_grid, n_jobs=4)
grid_result = grid.fit(A_train, b_train)



print("\n Bass - Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.utils.np_utils import to_categorical
# from sklearn.metrics import r2_score


# def R2(y_true, y_pred):
#     return r2_score(y_true, y_pred)

# label = 'k'
# x_train = (Xs_train[label])
# Y_train = (y_train[label][:,0])
# Y_test = (y_test[label][:,0])
# x_test = (Xs_test[label])

# model = Sequential()
# model.add(Dense(64, input_dim=4, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(4, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(1, activation='linear'))

# model.compile(loss='mean_absolute_error',
#               optimizer='adam',
#               metrics=['accuracy'])

# model.fit(x_train, Y_train, epochs=160, verbose=1, validation_split=0.05, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0)



In [ ]:
# y_pred = model.predict(x_test)
# r2_score(Y_test, y_pred)

In [ ]:
# from sklearn.multioutput import MultiOutputRegressor

# max_depth = 30
# regr_multirf = MultiOutputRegressor(RandomForestRegressor(max_depth=max_depth,
#                                                           random_state=0))
# regr_multirf.fit(x_train, Y_train)

# regr_rf = RandomForestRegressor(n_estimators=2000, max_depth=None, random_state=None)
# regr_rf.fit(x_train, Y_train)

# # Predict on new data
# y_multirf = regr_multirf.predict(x_test)
# y_rf = regr_rf.predict(x_test)

In [ ]:
# r2_score(Y_test, y_multirf)

In [ ]:
# r2_score(Y_test, y_rf)

In [ ]:
# Bass

#%matplotlib inline

hyp.plot(dfBass,
         ['o','^'],
         group = y[kInstruments[0]],
         normalize = 'within',
         legend = ['raw','stem'],
         ndims = 2,
         #explore = True,
         #animate = True, duration = 30, tail_duration = 10
        )

In [ ]:
# Guitar

#%matplotlib inline

hyp.plot(dfGuitar,
         ['o','^'],
         group = y[kInstruments[1]],
         normalize = 'within',
         legend = ['raw','stem'],
         ndims = 2,
         #explore = True,
         #animate = True, duration = 30, tail_duration = 10
        )

In [ ]:
# Vocal

#%matplotlib inline

hyp.plot(dfVocal,
         ['o','^'],
         group = y[kInstruments[2]],
         normalize = 'within',
         legend = ['raw','stem'],
         ndims = 2,
         #explore = True,
         #animate = True, duration = 30, tail_duration = 10
        )


In [ ]:
# Keys

#%matplotlib inline

hyp.plot(dfKeys,
         ['o','^'],
         group = y[kInstruments[3]],
         normalize = 'within',
         legend = ['raw','stem'],
         ndims = 2,
         #explore = True,
         #animate = True, duration = 30, tail_duration = 10
        )

In [ ]:
#Quantitative analysis:   
#%%

#Random Forest Classifier

def RandomForestC(data):
    for k in kInstruments:
        A = data[k][:]

        startTime = datetime.now() 

        #quantitative testing with a RF classifier, score is summaried over N epoch
        _Vscore = []
        epoch = 10                                        
        X_train, X_test, y_train, y_test = train_test_split(A, y[k],
                                                           train_size=0.8,
                                                           random_state=None)

        for i in range(epoch):

            #    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train,
            #                                                    train_size=0.8,
            #                                                    random_state=None)
            forest = RandomForestClassifier(n_estimators=2000, warm_start=False,
                                               oob_score=False, max_features=0.3, random_state=None)
            kfold = KFold(n_splits=5)

            for train, test in kfold.split(X_train):

                forest.fit(X_train[train], y_train[train])
                _Vscore.append(forest.score(X_train[test], y_train[test]))   
        print '\n %s - RF Validation R2 score mean: %f \n' % (k, np.mean(_Vscore))      
        print '\n %s - RF Validation R2 score std: %f \n' % (k, np.std(_Vscore))
        print '\n %s - RF Test R2 score: %f \n' % (k, forest.score(X_test, y_test))    
        print '\nExecuted in: %s. \n ' % (str(datetime.now() - startTime))  



In [ ]:
#SVC
def SupportVectorC(data):
    for k in kInstruments:
        A = data[k][:]

        startTime = datetime.now() 

        #quantitative testing with a RF classifier, score is summaried over N epoch
        _Vscore = []
        epoch = 100                                       
        X_train, X_test, y_train, y_test = train_test_split(A, y[k],
                                                           train_size=0.8,
                                                           random_state=None)

        for i in range(epoch):

            svc = SVC(kernel='rbf', C=1, gamma='auto')
            kfold = KFold(n_splits=5)

            for train, test in kfold.split(X_train):

                svc.fit(X_train[train], y_train[train])
                _Vscore.append(svc.score(X_train[test], y_train[test]))
        print '\n %s - SVC Validation R2 score mean: %f \n' % (k, np.mean(_Vscore))      
        print '\n %s - SVC Validation R2 score std: %f \n' % (k, np.std(_Vscore))
        print '\n %s - SVC Test R2 score: %f \n' % (k, svc.score(X_test, y_test))  
        
        
        print '\nExecuted in: %s. \n ' % (str(datetime.now() - startTime))  

In [ ]:
#LGC
def LogisticRegressionC(data):
    for k in kInstruments:
        A = data[k][:]
        startTime = datetime.now() 

        #quantitative testing with a RF classifier, score is summaried over N epoch
        _Vscore = []
        epoch = 100                                       
        X_train, X_test, y_train, y_test = train_test_split(A, y[k],
                                                           train_size=0.8,
                                                           random_state=None)

        for i in range(epoch):

            clf = LogisticRegression(C=1,  tol=0.01)
            
            kfold = KFold(n_splits=5)

            for train, test in kfold.split(X_train):

                clf.fit(X_train[train], y_train[train])
                _Vscore.append(clf.score(X_train[test], y_train[test]))

        print '\n %s - LG Validation R2 score mean: %f \n' % (k, np.mean(_Vscore))      
        print '\n %s - LG Validation R2 score std: %f \n' % (k, np.std(_Vscore))
        print '\n %s - LG Test R2 score: %f \n' % (k, clf.score(X_test, y_test))    
        print '\nExecuted in: %s. \n ' % (str(datetime.now() - startTime))   


In [ ]:
print '\n Prediction RFC: \n'
RandomForestC(XPrediction)
print '\n Interpretation RFC: \n'
RandomForestC(XInterpretation)
print '\n Preselected RFC: \n'
RandomForestC(XPreSelected)
print '\n All RFC: \n'
RandomForestC(XAll)





In [ ]:
print '\n Prediction SVC: \n'
SupportVectorC(XPrediction)
print '\n Interpretation SVC: \n'
SupportVectorC(XInterpretation)
print '\n Preselected SVC: \n'
SupportVectorC(XPreSelected)
print '\n All SVC: \n'
SupportVectorC(XAll)




In [ ]:
print '\n Prediction LG: \n'
LogisticRegressionC(XPrediction)
print '\n Interpretation LG: \n'
LogisticRegressionC(XInterpretation)
print '\n Preselected LG: \n'
LogisticRegressionC(XPreSelected)
print '\n All LG: \n'
LogisticRegressionC(XAll)

In [ ]:
#Quantitative analysis:   
#%%

#Random Forest Classifier

def Classifiers(A_train, A_test, b_train, b_test):
    #for k in kInstruments:
        #A = data[k][:]

        startTime = datetime.now() 

        #quantitative testing with a RF classifier, score is summaried over N epoch
        _VRF = []
        _VSV = []
        _VLG = []
        epoch = 100                                        
        X_train = A_train[:]
        X_test = A_test[:]
        y_train = b_train[:]
        y_test = b_test[:]
        
        
        forest = RandomForestClassifier(n_estimators=500, warm_start=False,
                                               oob_score=False, max_features='sqrt', random_state=None)
            
        clf = LogisticRegression(C=1,  tol=0.01)
        svc = SVC(kernel='rbf', C=1, gamma='auto')
        for i in range(epoch):

            #    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train,
            #                                                    train_size=0.8,
            #                                                    random_state=None)
            
            
            kfold = KFold(n_splits=5)

            for train, test in kfold.split(X_train):

                forest.fit(X_train[train], y_train[train])
                _VRF.append(forest.score(X_train[test], y_train[test]))  
                
                clf.fit(X_train[train], y_train[train])
                _VLG.append(clf.score(X_train[test], y_train[test]))
                
                svc.fit(X_train[train], y_train[train])
                _VSV.append(svc.score(X_train[test], y_train[test]))
                
        print '\n %s - RF Validation mean accuracy: %f \n' % (kInstruments[k], np.mean(_VRF))      
        print '\n %s - RF Validation std mean accuracy: %f \n' % (kInstruments[k], np.std(_VRF))
        print '\n %s - RF Test mean accuracy: %f \n' % (kInstruments[k], forest.score(X_test, y_test))    
        
        print '\n %s - SVC Validation mean accuracy: %f \n' % (kInstruments[k], np.mean(_VSV))      
        print '\n %s - SVC Validation std mean accuracy: %f \n' % (kInstruments[k], np.std(_VSV))
        print '\n %s - SVC Test mean accuracy: %f \n' % (kInstruments[k], svc.score(X_test, y_test))  
        
        print '\n %s - LG Validation mean accuracy: %f \n' % (kInstruments[k], np.mean(_VLG))      
        print '\n %s - LG Validation std mean accuracy: %f \n' % (kInstruments[k], np.std(_VLG))
        print '\n %s - LG Test mean accuracy: %f \n' % (kInstruments[k], clf.score(X_test, y_test))
        
        print '\nExecuted in: %s. \n ' % (str(datetime.now() - startTime)) 


In [ ]:
k=0

XP_train, XP_test, XI_train, XI_test, XAll_train, XAll_test, y_train, y_test = train_test_split(XPrediction[kInstruments[k]],
                                                                                                                                      XInterpretation[kInstruments[k]],
                                                                                                                                      
                                                                                                                                      XAll[kInstruments[k]],                                                                                                                                      
                                                                                                                                      y[kInstruments[k]],
                                                                                                                                      train_size=0.8,
                                                                                                                                      random_state=None)




print '\n %s \n'% kInstruments[k]
print '\n Prediction Features: \n'
Classifiers(XP_train, XP_test, y_train, y_test)
print '\n Interpretation Features: \n'
Classifiers(XI_train, XI_test, y_train, y_test)

print '\n All Features: \n'
Classifiers(XAll_train, XAll_test,y_train, y_test)                                                                                                                       

In [ ]:
k=1

XP_train, XP_test, XI_train, XI_test, XAll_train, XAll_test, y_train, y_test = train_test_split(XPrediction[kInstruments[k]],
                                                                                                                                      XInterpretation[kInstruments[k]],
                                                                                                                                      
                                                                                                                                      XAll[kInstruments[k]],                                                                                                                                      
                                                                                                                                      y[kInstruments[k]],
                                                                                                                                      train_size=0.8,
                                                                                                                                      random_state=None)




print '\n %s \n'% kInstruments[k]
print '\n Prediction Features: \n'
Classifiers(XP_train, XP_test, y_train, y_test)
print '\n Interpretation Features: \n'
Classifiers(XI_train, XI_test, y_train, y_test)

print '\n All Features: \n'
Classifiers(XAll_train, XAll_test,y_train, y_test)                                                                                                                       

In [ ]:
k=2

XP_train, XP_test, XI_train, XI_test, XAll_train, XAll_test, y_train, y_test = train_test_split(XPrediction[kInstruments[k]],
                                                                                                                                      XInterpretation[kInstruments[k]],
                                                                                                                                      
                                                                                                                                      XAll[kInstruments[k]],                                                                                                                                      
                                                                                                                                      y[kInstruments[k]],
                                                                                                                                      train_size=0.8,
                                                                                                                                      random_state=None)




print '\n %s \n'% kInstruments[k]
print '\n Prediction Features: \n'
Classifiers(XP_train, XP_test, y_train, y_test)
print '\n Interpretation Features: \n'
Classifiers(XI_train, XI_test, y_train, y_test)

print '\n All Features: \n'
Classifiers(XAll_train, XAll_test,y_train, y_test)                                                                                                                       

In [ ]:
k=3

XP_train, XP_test, XI_train, XI_test, XAll_train, XAll_test, y_train, y_test = train_test_split(XPrediction[kInstruments[k]],
                                                                                                                                      XInterpretation[kInstruments[k]],
                                                                                                                                      
                                                                                                                                      XAll[kInstruments[k]],                                                                                                                                      
                                                                                                                                      y[kInstruments[k]],
                                                                                                                                      train_size=0.8,
                                                                                                                                      random_state=None)




print '\n %s \n'% kInstruments[k]
print '\n Prediction Features: \n'
Classifiers(XP_train, XP_test, y_train, y_test)
print '\n Interpretation Features: \n'
Classifiers(XI_train, XI_test, y_train, y_test)

print '\n All Features: \n'
Classifiers(XAll_train, XAll_test,y_train, y_test)                                                                                                                       

In [ ]:
 keys-all3 


 Prediction Features: 


 keys-all3 - RF Validation mean accuracy: 0.638657 


 keys-all3 - RF Validation std mean accuracy: 0.054666 


 keys-all3 - RF Test mean accuracy: 0.692308 


 keys-all3 - SVC Validation mean accuracy: 0.711429 


 keys-all3 - SVC Validation std mean accuracy: 0.067586 


 keys-all3 - SVC Test mean accuracy: 0.653846 


 keys-all3 - LG Validation mean accuracy: 0.680952 


 keys-all3 - LG Validation std mean accuracy: 0.106053 


 keys-all3 - LG Test mean accuracy: 0.538462 


Executed in: 0:11:45.990101. 
 

 Interpretation Features: 


 keys-all3 - RF Validation mean accuracy: 0.676381 


 keys-all3 - RF Validation std mean accuracy: 0.099669 


 keys-all3 - RF Test mean accuracy: 0.730769 


 keys-all3 - SVC Validation mean accuracy: 0.700952 


 keys-all3 - SVC Validation std mean accuracy: 0.091270 


 keys-all3 - SVC Test mean accuracy: 0.653846 


 keys-all3 - LG Validation mean accuracy: 0.739524 


 keys-all3 - LG Validation std mean accuracy: 0.096228 


 keys-all3 - LG Test mean accuracy: 0.653846 


Executed in: 0:11:49.666839. 
 

 All Features: 


 keys-all3 - RF Validation mean accuracy: 0.644919 


 keys-all3 - RF Validation std mean accuracy: 0.060663 


 keys-all3 - RF Test mean accuracy: 0.653846 


 keys-all3 - SVC Validation mean accuracy: 0.431905 


 keys-all3 - SVC Validation std mean accuracy: 0.062807 


 keys-all3 - SVC Test mean accuracy: 0.461538 


 keys-all3 - LG Validation mean accuracy: 0.441905 


 keys-all3 - LG Validation std mean accuracy: 0.052025 


 keys-all3 - LG Test mean accuracy: 0.461538 


Executed in: 0:14:02.908636. 

In [ ]:
0.6

In [ ]:
(0.607-0.714)+(0.606-0.788)+(0.666-0.833)+(0.653-0.692)

In [ ]:
0.495/4